In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
from IPython.display import YouTubeVideo
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 

from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-notifications")

try:
    df = pd.read_csv (r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')

except:
    PATH = "/Users/jordandass/Desktop/Drivers/chromedriver"

    driver = webdriver.Chrome(PATH, chrome_options = options)

    driver.get("https://www.fplanalytics.com/playerStatus.html")

    driver.implicitly_wait(15)
    dwnld_button = driver.find_element_by_xpath("/html/body/section[2]/div/input")
    webdriver.ActionChains(driver).move_to_element(dwnld_button).click(dwnld_button).perform()

    time.sleep(15)


    df = pd.read_csv (r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')


    send2trash(r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')

    driver.quit()

name_list = []
cost_list = []
total_points_list = []
position_list = []
ppg_list = []
team_list = []
points_cost_list = []
clean_name_list = []

time_now = re.findall('[0-9]{4}-[0-9]+-[0-9]+', str(datetime.now()))[0]

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

for num in range(len(df)):
    if int(df.iloc[num, 6]) != 0:
        if str(df.iloc[num, 4]) != 'NotAvail':
            name_list.append(df.iloc[num, 0])
            cost_list.append(df.iloc[num, 3])
            total_points_list.append(df.iloc[num, 6])
            position_list.append(df.iloc[num, 2])
            ppg_list.append(df.iloc[num, 8])
            team_list.append(df.iloc[num, 1])
            points_cost_list.append(format(float(df.iloc[num, 6])/float(df.iloc[num, 3]),'.2f'))
            clean_name_list.append(strip_accents(str(df.iloc[num, 0]).lower()))
            
prices_df = pd.read_csv('prices.csv')
if list(prices_df.iloc[:,-1]) != cost_list or time_now in list(prices_df.columns):
    prices_df[time_now] = cost_list
    prices_df.to_csv('prices.csv', index = False)


fpl_players = pd.DataFrame({'Players':name_list, 'Costs':cost_list, 'Total points':total_points_list, 'Positions':position_list, 'PPG':ppg_list, 'Team':team_list, 'Points per mil':points_cost_list, 'Clean name':clean_name_list})

fpl_players.to_csv('fpl_players.csv')
cost_effective = fpl_players.sort_values(by = 'Points per mil', ascending = False)
cost_effective.to_csv('cost_effective.csv')

gk_df = []
def_df = []
mid_df = []
fwd_df = []

for position in ['GKP','DEF','MID','FWD']:
    for num in range(len(fpl_players)):
        if fpl_players.iloc[num, 3] == position:
            if position == 'GKP':
                gk_df.append(list(fpl_players.iloc[num,]))
            elif position == 'DEF':
                def_df.append(list(fpl_players.iloc[num,]))
            elif position == 'MID':
                mid_df.append(list(fpl_players.iloc[num,]))
            elif position == 'FWD':
                fwd_df.append(list(fpl_players.iloc[num,]))
                

gk_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = gk_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
def_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = def_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
mid_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = mid_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
fwd_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = fwd_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])

best_df = pd.concat([gk_df, def_df, mid_df, fwd_df]).sort_values(by = 'Points per mil', ascending = False)
best_df.to_csv("best.csv")


gk_prices = sorted(list(set(list(gk_df.iloc[:,1]))))

def_prices = sorted(list(set(list(def_df.iloc[:,1]))))
    
mid_prices = sorted(list(set(list(mid_df.iloc[:,1]))))

fwd_prices = sorted(list(set(list(fwd_df.iloc[:,1]))))

all_price_quantity = {}
gk_price_quantity = {}
def_price_quantity = {}
mid_price_quantity = {}
fwd_price_quantity = {}


for price_list in [gk_prices, def_prices, mid_prices, fwd_prices]:
    for price in price_list:
        if price_list == gk_prices:
            gk_price_quantity[price] = len(gk_df[gk_df['Costs'] == price])
        elif price_list == def_prices:
            def_price_quantity[price] = len(def_df[def_df['Costs'] == price])
        elif price_list == mid_prices:
            mid_price_quantity[price] = len(mid_df[mid_df['Costs'] == price])
        elif price_list == fwd_prices:
            fwd_price_quantity[price] = len(fwd_df[fwd_df['Costs'] == price])    
            
    all_price_quantity['gk'] = gk_price_quantity
    all_price_quantity['def'] = def_price_quantity
    all_price_quantity['mid'] = mid_price_quantity
    all_price_quantity['fwd'] = fwd_price_quantity
print(all_price_quantity)

    




{'gk': {4.0: 3, 4.5: 7, 4.6: 2, 4.8: 1, 4.9: 1, 5.0: 4, 5.1: 2, 5.4: 1, 5.5: 1, 5.6: 1, 6.0: 3}, 'def': {3.9: 1, 4.0: 7, 4.2: 1, 4.3: 11, 4.4: 29, 4.5: 21, 4.6: 5, 4.7: 1, 4.8: 6, 4.9: 12, 5.0: 16, 5.1: 1, 5.2: 1, 5.3: 3, 5.4: 6, 5.5: 5, 5.6: 1, 5.7: 1, 6.0: 2, 6.1: 2, 6.6: 1, 7.0: 1, 7.5: 1}, 'mid': {4.4: 2, 4.5: 27, 4.6: 3, 4.8: 5, 4.9: 16, 5.0: 17, 5.2: 2, 5.3: 14, 5.4: 15, 5.5: 11, 5.6: 1, 5.7: 2, 5.8: 11, 5.9: 11, 6.0: 7, 6.3: 3, 6.4: 6, 6.5: 1, 6.6: 2, 6.7: 2, 6.9: 3, 7.0: 1, 7.1: 1, 7.2: 1, 7.4: 2, 7.6: 1, 7.7: 1, 7.8: 1, 7.9: 2, 8.0: 1, 8.2: 1, 8.7: 1, 9.0: 1, 10.0: 1, 10.7: 1, 11.9: 3, 12.6: 1}, 'fwd': {4.5: 2, 4.9: 2, 5.0: 2, 5.3: 6, 5.4: 5, 5.5: 2, 5.8: 2, 5.9: 2, 6.0: 3, 6.3: 3, 6.5: 2, 6.8: 2, 6.9: 1, 7.0: 1, 7.1: 1, 7.3: 2, 7.4: 1, 7.5: 1, 7.6: 1, 7.8: 1, 7.9: 1, 8.0: 2, 8.3: 1, 8.4: 1, 8.6: 2, 8.8: 1, 9.9: 1, 10.4: 1, 11.7: 1, 12.1: 1, 12.7: 1}}


In [5]:
import unicodedata
from collections import defaultdict
import numpy as np
from itertools import product

def team_build(budget, my_team):
    x = None

    my_team = my_team
    my_team_list = []
    for t in range(len(my_team)):
        for player in my_team[t]:
            if player != None:
                my_team_list.append(player)


    goalkeepers = my_team[0]
    defenders = my_team[1]
    midfielders = my_team[2]
    fowards = my_team[3]

    fill_gkp = 0
    fill_def = 0
    fill_mid = 0
    fill_fwd = 0
    budget = float(budget)
    teams = defaultdict(int)

    for position in [goalkeepers, defenders, midfielders, fowards]:
        for player in position:
            if position == goalkeepers:
                if player == x:
                    fill_gkp += 1
                else:
                    budget -= float(gk_df[gk_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(gk_df[gk_df['Clean name'] == player.lower()]['Team'])[0]] += 1

            if position == defenders:
                if player == x:
                    fill_def += 1
                else:
                    budget -= float(def_df[def_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(def_df[def_df['Clean name'] == player.lower()]['Team'])[0]] += 1

            if position == midfielders:
                if player == x:
                    fill_mid += 1
                else:
                    budget -= float(mid_df[mid_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(mid_df[mid_df['Clean name'] == player.lower()]['Team'])[0]] += 1

            if position == fowards:
                if player == x:
                    fill_fwd += 1
                else:
                    budget -= float(fwd_df[fwd_df['Clean name'] == player.lower()]['Costs'])
                    teams[list(fwd_df[fwd_df['Clean name'] == player.lower()]['Team'])[0]] += 1

    budget = format(float(budget), '.1f')
    print(budget)
    print(list(teams))
    print(my_team_list)

    full_teams = []
    for team in teams.keys():
        if teams[team] >= 3:
            full_teams.append(team)

    potential_gk = gk_df[~gk_df['Clean name'].isin(my_team_list) & ~gk_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)
    potential_def = def_df[~def_df['Clean name'].isin(my_team_list) & ~def_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)
    potential_mid = mid_df[~mid_df['Clean name'].isin(my_team_list) & ~mid_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)
    potential_fwd = fwd_df[~fwd_df['Clean name'].isin(my_team_list) & ~fwd_df['Team'].isin(my_team_list)].sort_values(by = 'Costs', ascending = True)

    for fill in [fill_gkp, fill_def, fill_mid, fill_fwd]:
        min_gkp_cost = 0
        min_def_cost = 0
        min_mid_cost = 0
        min_fwd_cost = 0
        for num in range(fill):
            if fill == fill_gkp:
                min_gkp_cost += float(gk_df.iloc[num, 1])
            if fill == fill_def:
                min_def_cost += float(def_df.iloc[num, 1])
            if fill == fill_mid:
                min_mid_cost += float(mid_df.iloc[num, 1])
            if fill == fill_fwd:
                min_fwd_cost += float(fwd_df.iloc[num, 1])

    gk_comb = list(combinations(list(range(len(potential_gk))), fill_gkp))
    def_comb = list(combinations(list(range(len(potential_def))), fill_def))
    mid_comb = list(combinations(list(range(len(potential_mid))), fill_mid))
    fwd_comb = list(combinations(list(range(len(potential_fwd))), fill_fwd))

    gk_comb_poss = []
    def_comb_poss = []
    mid_comb_poss = []
    fwd_comb_poss = []
    for pos_comb in [gk_comb, def_comb, mid_comb, fwd_comb]:
        for comb in pos_comb:
            price = 0
            if pos_comb == gk_comb:
                for num in comb:
                    price += float(potential_gk.iloc[num, 1])
                if (float(budget) - price - min_def_cost - min_mid_cost - min_fwd_cost) >= 0:
                    gk_comb_poss.append(comb)
            elif pos_comb == def_comb:
                for num in comb:
                    price += float(potential_def.iloc[num, 1])
                if (float(budget) - price - min_gkp_cost - min_mid_cost - min_fwd_cost) >= 0:
                    def_comb_poss.append(comb)
            elif pos_comb == mid_comb:
                for num in comb:
                    price += float(potential_mid.iloc[num, 1])
                if (float(budget) - price - min_def_cost - min_gkp_cost - min_fwd_cost) >= 0:
                    mid_comb_poss.append(comb)
            elif pos_comb == gk_comb:
                for num in comb:
                    price += float(potential_fwd.iloc[num, 1])
                if (float(budget) - price - min_def_cost - min_mid_cost - min_gkp_cost) >= 0:
                    fwd_comb_poss.append(comb)


    def cartesian_product(arr1, arr2, arr3, arr4):
        for arr in [arr1, arr2, arr3, arr4]:
            if arr != []:
                arr = None
        return list(product(arr1, arr2, arr3, arr4))

    if gk_comb_poss == [] or gk_comb_poss == [(), ()]:
        gk_comb_poss = ['No gk']
    if def_comb_poss == [] or def_comb_poss == [(), ()]:
        def_comb_poss = ['No def']
    if mid_comb_poss == [] or mid_comb_poss == [(), ()]:
        mid_comb_poss = ['No mid']
    if fwd_comb_poss == [] or fwd_comb_poss == [(), ()]:
        fwd_comb_poss = ['No fwd']

    score_player = defaultdict(list)
    for pos_comb in cartesian_product(gk_comb_poss, def_comb_poss, mid_comb_poss, fwd_comb_poss):
        player_comb_num = 0
        highest_score = 0
        players = []
        for player_comb in pos_comb:
            for player_num in player_comb:
                if type(player_num) == int:
                    if player_comb == pos_comb[0]:
                        highest_score += float(potential_gk.iloc[player_num , 2])
                        players += [potential_gk.iloc[player_num, 0]]

                    elif player_comb == pos_comb[1]:
                        highest_score += float(potential_def.iloc[player_num, 2])
                        players += [potential_def.iloc[player_num, 0]]

                    elif player_comb == pos_comb[2]:
                        highest_score += float(potential_mid.iloc[player_num, 2])
                        players += [potential_mid.iloc[player_num, 0]]

                    elif player_comb == pos_comb[3]:
                        highest_score += float(potential_fwd.iloc[player_num, 2])
                        players += [potential_fwd.iloc[player_num, 0]]
                    player_comb_num += 1
        score_player[highest_score] += [players]

    final_comb = defaultdict(list)
    for score in score_player.keys():
        for player_comb in score_player[score]:
            spent = 0
            for player in player_comb:
                spent += list(best_df[best_df['Players'] == player]['Costs'])[0]
            if float(spent) <= float(budget):
                final_comb[score] += ([player_comb] + [spent])
    for key in sorted(list(final_comb.keys()), reverse = True):
        print(key)
        print(final_comb[key])
        print('')
        
x = None
team_build(100.8, 
           [['sanchez', x],
          ['rudiger', 'white', 'alexander-arnold', 'livramento', x],
          ['salah', 'gray', 'sarr', 'jota', 'douglas luiz'],
          ['ronaldo', 'lukaku','Archer']])

8.5
['BHA', 'CHE', 'ARS', 'LIV', 'SOU', 'EVE', 'WAT', 'AVL', 'MUN']
['sanchez', 'rudiger', 'white', 'alexander-arnold', 'livramento', 'salah', 'gray', 'sarr', 'jota', 'douglas luiz', 'ronaldo', 'lukaku', 'Archer']
47.0
[['Guaita', 'Tsimikas'], 8.5]

43.0
[['McCarthy', 'Tsimikas'], 8.5]

41.0
[['Ramsdale', 'Tsimikas'], 8.5]

35.0
[['Foster', 'Duffy'], 8.3]

34.0
[['Guaita', 'Manquillo'], 8.5]

33.0
[['Darlow', 'Tsimikas'], 8.5]

32.0
[['Guaita', 'Amartey'], 8.5, ['Woodman', 'Tsimikas'], 8.5]

31.0
[['Bachmann', 'Tsimikas'], 8.5]

30.0
[['Steer', 'Duffy'], 8.3, ['Begović', 'Duffy'], 8.3, ['McCarthy', 'Manquillo'], 8.5, ['Krul', 'Tsimikas'], 8.5]

29.0
[['Foster', 'Tsimikas'], 8.0, ['Foster', 'Ajer'], 8.5, ['Raya', 'Omobamidele'], 8.5]

28.0
[['Foster', 'Henry'], 8.5, ['Ramsdale', 'Manquillo'], 8.5, ['McCarthy', 'Amartey'], 8.5]

27.0
[['Guaita', 'Williams'], 8.5]

26.0
[['Guaita', 'Omobamidele'], 8.4, ['Guaita', 'Hoever'], 8.5]

24.0
[['Foster', 'Kilman'], 8.5, ['Foster', 'Coady'], 8.5, 

In [28]:
[[2]]+[3]

[[2], 3]

In [2]:
"""                
potential_teams = {}
for gk_p in gk_comb_poss:
    score = 0
    team = []
    for p in gk_p:
        team.append(gk_df.iloc[p, 0])
        score += float(gk_df.iloc[p, 2])
        for def_p in def_comb_poss:
            for p in def_p:
                team.append(def_df.iloc[p, 0])
                score += float(def_df.iloc[p, 2])
                for mid_p in mid_comb_poss:
                    for p in mid_p:
                        team.append(mid_df.iloc[p, 0])
                        score += float(mid_df.iloc[p, 2])
                        for fwd_p in fwd_comb_poss:
                            for p in fwd_p:
                                team.append(fwd_df.iloc[p, 0])
                                score += float(fwd_df.iloc[p, 2])
                                """



from collections import defaultdict
from itertools import combinations_with_replacement
import itertools

gk_comb = list(combinations_with_replacement(gk_prices, 2))
def_comb = list(combinations_with_replacement(def_prices, 5))
mid_comb = list(combinations_with_replacement(mid_prices, 5))
fwd_comb = list(combinations_with_replacement(fwd_prices, 3))

all_price_quantity = {}
gk_price_quantity = {}
def_price_quantity = {}
mid_price_quantity = {}
fwd_price_quantity = {}


for price_list in [gk_prices, def_prices, mid_prices, fwd_prices]:
    for price in price_list:
        if price_list == gk_prices:
            gk_price_quantity[price] = len(gk_df[gk_df['Costs'] == price])
        elif price_list == def_prices:
            def_price_quantity[price] = len(def_df[def_df['Costs'] == price])
        elif price_list == mid_prices:
            mid_price_quantity[price] = len(mid_df[mid_df['Costs'] == price])
        elif price_list == fwd_prices:
            fwd_price_quantity[price] = len(fwd_df[fwd_df['Costs'] == price])    
            
    all_price_quantity['gk'] = gk_price_quantity
    all_price_quantity['def'] = def_price_quantity
    all_price_quantity['mid'] = mid_price_quantity
    all_price_quantity['fwd'] = fwd_price_quantity
print(all_price_quantity)

possible_comb_array = []

possible_gk_comb = []
for comb in gk_comb:
    price_check = defaultdict(int)
    for price in comb:
        price_check[price] += 1
    check = True
    for price in comb:
        if price_check[price] <= all_price_quantity['gk'][price]:
            continue
        else:
            check = False
    if check == True:
        possible_gk_comb.append(comb)
print(len(gk_comb))
print(len(possible_gk_comb))
possible_comb_array.append(possible_gk_comb)

possible_def_comb = []
for comb in def_comb:
    price_check = defaultdict(int)
    for price in comb:
        price_check[price] += 1
    check = True
    for price in comb:
        if price_check[price] <= all_price_quantity['def'][price]:
            continue
        else:
            check = False
    if check == True:
        possible_def_comb.append(comb)
print(len(def_comb))
print(len(possible_def_comb))
possible_comb_array.append(possible_def_comb)



possible_mid_comb = []
for comb in mid_comb:
    price_check = defaultdict(int)
    for price in comb:
        price_check[price] += 1
    check = True
    for price in comb:
        if price_check[price] <= all_price_quantity['mid'][price]:
            continue
        else:
            check = False
    if check == True:
        possible_mid_comb.append(comb)
print(len(mid_comb))
print(len(possible_mid_comb))
possible_comb_array.append(possible_mid_comb)



possible_fwd_comb = []
for comb in fwd_comb:
    price_check = defaultdict(int)
    for price in comb:
        price_check[price] += 1
    check = True
    for price in comb:
        if price_check[price] <= all_price_quantity['fwd'][price]:
            continue
        else:
            check = False
    if check == True:
        possible_fwd_comb.append(comb)
print(len(fwd_comb))
print(len(possible_fwd_comb))
possible_comb_array.append(possible_fwd_comb)

formations = [[4,4,2],[4,3,3],[3,4,3],[3,5,2],[5,3,2],[5,2,3]]
list_of_teams = []
for formation in formations:
    def_min_cost = 0
    for num in range(1, 5 - formation[0] + 1):
        def_min_cost += float(def_df.iloc[num, 1])
    
    mid_min_cost = 0
    for num in range(1, 5 - formation[1] + 1):
        mid_min_cost += float(mid_df.iloc[num, 1])
        
    fwd_min_cost = 0
    for num in range(1, 3 - formation[2] + 1):
        fwd_min_cost += float(fwd_df.iloc[num, 1])
    budget = 100 - def_min_cost - mid_min_cost - fwd_min_cost
    print(budget)

            
    


    

{'gk': {4.0: 3, 4.5: 7, 4.6: 2, 4.8: 1, 4.9: 1, 5.0: 5, 5.1: 1, 5.4: 1, 5.5: 1, 5.6: 1, 6.0: 3}, 'def': {3.9: 1, 4.0: 7, 4.2: 1, 4.3: 11, 4.4: 28, 4.5: 22, 4.6: 5, 4.7: 1, 4.8: 6, 4.9: 12, 5.0: 16, 5.2: 2, 5.3: 2, 5.4: 6, 5.5: 6, 5.6: 1, 5.7: 1, 5.9: 1, 6.0: 1, 6.1: 2, 6.6: 1, 7.0: 1, 7.6: 1}, 'mid': {4.4: 2, 4.5: 27, 4.6: 3, 4.8: 3, 4.9: 17, 5.0: 18, 5.2: 2, 5.3: 12, 5.4: 17, 5.5: 11, 5.6: 1, 5.7: 1, 5.8: 11, 5.9: 11, 6.0: 8, 6.2: 1, 6.3: 2, 6.4: 6, 6.5: 2, 6.6: 1, 6.7: 1, 6.8: 1, 6.9: 3, 7.0: 1, 7.2: 2, 7.4: 2, 7.7: 2, 7.8: 1, 7.9: 2, 8.0: 1, 8.2: 1, 8.7: 1, 9.0: 1, 10.0: 1, 10.8: 1, 11.9: 3, 12.6: 1}, 'fwd': {4.5: 2, 4.9: 2, 5.0: 2, 5.3: 5, 5.4: 6, 5.5: 2, 5.8: 2, 5.9: 2, 6.0: 3, 6.3: 3, 6.5: 2, 6.8: 2, 6.9: 1, 7.0: 1, 7.1: 1, 7.3: 1, 7.4: 2, 7.5: 1, 7.6: 1, 7.9: 3, 8.0: 1, 8.4: 2, 8.6: 2, 8.8: 1, 9.9: 1, 10.4: 1, 11.7: 1, 12.1: 1, 12.7: 1}}
66
60
80730
57967
749398
610546
4495
4107
87.1
87.1
87.6
87.5
86.6
86.6


In [ ]:
"Shit code"

formations = [[4,4,2],[4,3,3],[3,4,3],[3,5,2],[5,3,2],[5,2,3]]
list_of_teams = []
for formation in formations:
    def_min_cost = 0
    for num in range(1, 5 - formation[0] + 1):
        def_min_cost += float(def_df.iloc[num, 1])
    
    mid_min_cost = 0
    for num in range(1, 5 - formation[1] + 1):
        mid_min_cost += float(mid_df.iloc[num, 1])
        
    fwd_min_cost = 0
    for num in range(1, 3 - formation[2] + 1):
        fwd_min_cost += float(fwd_df.iloc[num, 1])
    budget = 100 - def_min_cost - mid_min_cost - fwd_min_cost
    print(budget)
    
    for gk_comb in list(combinations(range(len(gk_df)), 1)):
        for def_comb in list(combinations(range(len(def_df)), formation[0])):
            for mid_comb in list(combinations(range(len(mid_df)), formation[1])):
                for fwd_comb in list(combinations(range(len(fwd_df)), formation[2])):
                    total_cost = 0
                    
                    total_cost += gk_df.iloc[gk_comb[0], 1]
                    
                    for comb in [def_comb, mid_comb, fwd_comb]:
                        for num in comb:
                            if comb == def_comb:
                                total_cost += def_df.iloc[num, 1]
                            elif comb == mid_comb:
                                total_cost += mid_df.iloc[num, 1]
                            elif comb == fwd_comb:
                                total_cost += fwd_df.iloc[num, 1]
                    if total_cost <= budget:
                        
                        total_score = 0
                        for comb in [def_comb, mid_comb, fwd_comb]:
                            for num in comb:
                                if comb == def_comb:
                                    total_score += def_df.iloc[num, 2]
                                elif comb == mid_comb:
                                    total_score += mid_df.iloc[num, 2]
                                elif comb == fwd_comb:
                                    total_score += fwd_df.iloc[num, 2]
                                    
                        list_of_teams.append([total_score, total_cost, def_comb, mid_comb, fwd_comb, formation])
                        print(list_of_teams)
                    

In [38]:
import unicodedata
from collections import defaultdict
import numpy as np
from itertools import combinations
from itertools import product
x = None

my_team = [['sanchez', x],
          ['rudiger', 'white', 'alexander-arnold', 'livramento', x],
          ['salah', 'gray', 'sarr', 'gallagher', x],
          ['ronaldo', 'lukaku','antonio']]
my_team_list = []
for t in range(len(my_team)):
    for player in my_team[t]:
        if player != None:
            my_team_list.append(player)


goalkeepers = my_team[0]
defenders = my_team[1]
midfielders = my_team[2]
fowards = my_team[3]

fill_gkp = 0
fill_def = 0
fill_mid = 0
fill_fwd = 0
budget = 100.8
teams = defaultdict(int)

for position in [goalkeepers, defenders, midfielders, fowards]:
    for player in position:
        if position == goalkeepers:
            if player == x:
                fill_gkp += 1
            else:
                budget -= float(gk_df[gk_df['Clean name'] == player]['Costs'])
                teams[list(gk_df[gk_df['Clean name'] == player]['Team'])[0]] += 1
        
        if position == defenders:
            if player == x:
                fill_def += 1
            else:
                budget -= float(def_df[def_df['Clean name'] == player]['Costs'])
                teams[list(def_df[def_df['Clean name'] == player]['Team'])[0]] += 1
                
        if position == midfielders:
            if player == x:
                fill_mid += 1
            else:
                budget -= float(mid_df[mid_df['Clean name'] == player]['Costs'])
                teams[list(mid_df[mid_df['Clean name'] == player]['Team'])[0]] += 1
                
        if position == fowards:
            if player == x:
                fill_fwd += 1
            else:
                budget -= float(fwd_df[fwd_df['Clean name'] == player]['Costs'])
                teams[list(fwd_df[fwd_df['Clean name'] == player]['Team'])[0]] += 1
budget = format(budget, '.1f')
print(budget)
print(teams)
print(my_team_list)

full_teams = []
for team in teams.keys():
    if teams[team] >= 3:
        full_teams.append(team)

if x in goalkeepers:
    potential_gk = gk_df[~gk_df['Clean name'].isin(my_team_list) & ~gk_df['Team'].isin(full_teams)]
    pot_gk_comb = list(combinations(list(range(len(potential_gk))),fill_gkp))
else:
    pot_gk_comb = ['No gk']
    
if x in defenders:
    potential_def = def_df[~def_df['Clean name'].isin(my_team_list) & ~def_df['Team'].isin(full_teams)]
    pot_def_comb = list(combinations(list(range(len(potential_def))),fill_def))
else:
    pot_def_comb = ['No def']
    
if x in midfielders:
    potential_mid = mid_df[~mid_df['Clean name'].isin(my_team_list) & ~mid_df['Team'].isin(full_teams)]
    pot_mid_comb = list(combinations(list(range(len(potential_mid))),fill_mid))
else:
    pot_mid_comb = ['No mid']
    
if x in fowards:
    potential_fwd = fwd_df[~fwd_df['Clean name'].isin(my_team_list) & ~fwd_df['Team'].isin(full_teams)]
    pot_fwd_comb = list(combinations(list(range(len(potential_fwd))),fill_fwd))
else:
    pot_fwd_comb = ['No fwd']

     


def cartesian_product(arr1, arr2, arr3, arr4):
    return list(product(arr1, arr2, arr3, arr4))

print(cartesian_product(pot_gk_comb, pot_def_comb, pot_mid_comb, pot_fwd_comb))


   



    





IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
strip_accents('Śanchez')

'Sanchez'

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
from IPython.display import YouTubeVideo
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 
from selenium.webdriver.support.ui import Select


"""
PATH = "/Users/jordandass/Desktop/Drivers/chromedriver"

driver = webdriver.Chrome(PATH)

driver.get("https://www.fplanalytics.com/priceTable.html")

driver.implicitly_wait(15)

table_drop_down = Select(driver.find_element_by_name("data-table_length"))

driver.implicitly_wait(1)

table_drop_down.select_by_value('100')

"""
url = "https://www.fplanalytics.com/priceTable.html"
response = requests.get(url=url,)

soup = BeautifulSoup(response.content, 'lxml')
    
content = soup.find_all('//*[@id="data-table"]/tr')
print(content)

[]


In [14]:
PATH = "/Users/jordandass/Desktop/Drivers/chromedriver"

driver = webdriver.Chrome(PATH)

driver.get("https://www.facebook.com/login.php?skip_api_login=1&api_key=122423114856457&kid_directed_site=0&app_id=122423114856457&signed_next=1&next=https%3A%2F%2Fwww.facebook.com%2Fv2.12%2Fdialog%2Foauth%3Fclient_id%3D122423114856457%26redirect_uri%3Dhttps%253A%252F%252Fusers.premierleague.com%252Faccounts%252Ffacebook%252Flogin%252Fcallback%252F%26scope%3Demail%252Cpublic_profile%26response_type%3Dcode%26state%3DQtMczCbDnyUe%26ret%3Dlogin%26fbapp_pres%3D0%26logger_id%3Daa840df3-ed0c-4f23-b6cf-a1e1cedc439a%26tp%3Dunspecified&cancel_url=https%3A%2F%2Fusers.premierleague.com%2Faccounts%2Ffacebook%2Flogin%2Fcallback%2F%3Ferror%3Daccess_denied%26error_code%3D200%26error_description%3DPermissions%2Berror%26error_reason%3Duser_denied%26state%3DQtMczCbDnyUe%23_%3D_&display=page&locale=en_GB&pl_dbl=0")
driver.implicitly_wait(10)


facebook_username = driver.find_element_by_id("email")
facebook_username.send_keys("jordandass2000@gmail.com")

facebook_password = driver.find_element_by_id("pass")
facebook_password.send_keys("jordan2000")

driver.find_element_by_id("loginbutton").click()



In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import json
from IPython.display import YouTubeVideo
import pandas as pd
from send2trash import send2trash
from itertools import combinations
from itertools import product
from collections import defaultdict
import unicodedata
from datetime import datetime 

PATH = "/Users/jordandass/Desktop/Drivers/chromedriver"

driver = webdriver.Chrome(PATH)

driver.get("https://0e0c55ie39.execute-api.eu-central-1.amazonaws.com/default/fplAnalytics-DownloadPlayerStatusData")

time.sleep(10)

df = pd.read_csv (r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')

send2trash(r'/Users/jordandass/Downloads/fplAnalytics-playerStautsData.csv')

name_list = []
cost_list = []
total_points_list = []
position_list = []
ppg_list = []
team_list = []
points_cost_list = []
clean_name_list = []

time_now = re.findall('[0-9]{4}-[0-9]+-[0-9]+', str(datetime.now()))[0]

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

for num in range(len(df)):
    if int(df.iloc[num, 6]) != 0:
        if str(df.iloc[num, 4]) != 'NotAvail':
            name_list.append(df.iloc[num, 0])
            cost_list.append(df.iloc[num, 3])
            total_points_list.append(df.iloc[num, 6])
            position_list.append(df.iloc[num, 2])
            ppg_list.append(df.iloc[num, 8])
            team_list.append(df.iloc[num, 1])
            points_cost_list.append(format(float(df.iloc[num, 6])/float(df.iloc[num, 3]),'.2f'))
            clean_name_list.append(strip_accents(str(df.iloc[num, 0]).lower()))
            
prices_df = pd.read_csv('prices.csv')
if list(prices_df.iloc[:,-1]) != cost_list or time_now in list(prices_df.columns):
    prices_df[time_now] = cost_list
    prices_df.to_csv('prices.csv', index = False)


fpl_players = pd.DataFrame({'Players':name_list, 'Costs':cost_list, 'Total points':total_points_list, 'Positions':position_list, 'PPG':ppg_list, 'Team':team_list, 'Points per mil':points_cost_list, 'Clean name':clean_name_list})

fpl_players.to_csv('fpl_players.csv')
cost_effective = fpl_players.sort_values(by = 'Points per mil', ascending = False)
cost_effective.to_csv('cost_effective.csv')

gk_df = []
def_df = []
mid_df = []
fwd_df = []

for position in ['GKP','DEF','MID','FWD']:
    for num in range(len(fpl_players)):
        if fpl_players.iloc[num, 3] == position:
            if position == 'GKP':
                gk_df.append(list(fpl_players.iloc[num,]))
            elif position == 'DEF':
                def_df.append(list(fpl_players.iloc[num,]))
            elif position == 'MID':
                mid_df.append(list(fpl_players.iloc[num,]))
            elif position == 'FWD':
                fwd_df.append(list(fpl_players.iloc[num,]))
                

gk_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = gk_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
def_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = def_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
mid_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = mid_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])
fwd_df = pd.DataFrame(columns = ['Players', 'Costs', 'Total points', 'Positions', 'PPG','Team', 'Points per mil', 'Clean name'], data = fwd_df).sort_values(by = ['Costs', 'PPG'], ascending = [True, False])

best_df = pd.concat([gk_df, def_df, mid_df, fwd_df]).sort_values(by = 'Points per mil', ascending = False)
best_df.to_csv("best.csv")


gk_prices = sorted(list(set(list(gk_df.iloc[:,1]))))

def_prices = sorted(list(set(list(def_df.iloc[:,1]))))
    
mid_prices = sorted(list(set(list(mid_df.iloc[:,1]))))

fwd_prices = sorted(list(set(list(fwd_df.iloc[:,1]))))

all_price_quantity = {}
gk_price_quantity = {}
def_price_quantity = {}
mid_price_quantity = {}
fwd_price_quantity = {}


for price_list in [gk_prices, def_prices, mid_prices, fwd_prices]:
    for price in price_list:
        if price_list == gk_prices:
            gk_price_quantity[price] = len(gk_df[gk_df['Costs'] == price])
        elif price_list == def_prices:
            def_price_quantity[price] = len(def_df[def_df['Costs'] == price])
        elif price_list == mid_prices:
            mid_price_quantity[price] = len(mid_df[mid_df['Costs'] == price])
        elif price_list == fwd_prices:
            fwd_price_quantity[price] = len(fwd_df[fwd_df['Costs'] == price])    
            
    all_price_quantity['gk'] = gk_price_quantity
    all_price_quantity['def'] = def_price_quantity
    all_price_quantity['mid'] = mid_price_quantity
    all_price_quantity['fwd'] = fwd_price_quantity
print(all_price_quantity)
driver.quit()



{'gk': {4.0: 3, 4.5: 7, 4.6: 2, 4.8: 1, 4.9: 1, 5.0: 4, 5.1: 2, 5.4: 1, 5.5: 1, 5.6: 1, 6.0: 3}, 'def': {3.9: 1, 4.0: 7, 4.2: 1, 4.3: 11, 4.4: 29, 4.5: 21, 4.6: 5, 4.7: 1, 4.8: 6, 4.9: 12, 5.0: 16, 5.1: 1, 5.2: 1, 5.3: 3, 5.4: 6, 5.5: 5, 5.6: 1, 5.7: 1, 6.0: 2, 6.1: 2, 6.6: 1, 7.0: 1, 7.5: 1}, 'mid': {4.4: 2, 4.5: 27, 4.6: 3, 4.8: 5, 4.9: 16, 5.0: 17, 5.2: 2, 5.3: 14, 5.4: 15, 5.5: 11, 5.6: 1, 5.7: 2, 5.8: 11, 5.9: 11, 6.0: 7, 6.3: 3, 6.4: 6, 6.5: 1, 6.6: 2, 6.7: 2, 6.9: 3, 7.0: 1, 7.1: 1, 7.2: 1, 7.4: 2, 7.6: 1, 7.7: 1, 7.8: 1, 7.9: 2, 8.0: 1, 8.2: 1, 8.7: 1, 9.0: 1, 10.0: 1, 10.7: 1, 11.9: 3, 12.6: 1}, 'fwd': {4.5: 2, 4.9: 2, 5.0: 2, 5.3: 6, 5.4: 5, 5.5: 2, 5.8: 2, 5.9: 2, 6.0: 3, 6.3: 3, 6.5: 2, 6.8: 2, 6.9: 1, 7.0: 1, 7.1: 1, 7.3: 2, 7.4: 1, 7.5: 1, 7.6: 1, 7.8: 1, 7.9: 1, 8.0: 2, 8.3: 1, 8.4: 1, 8.6: 2, 8.8: 1, 9.9: 1, 10.4: 1, 11.7: 1, 12.1: 1, 12.7: 1}}
